In [1]:
import pandas as pd
import numpy as np
import  scipy.stats as st
import pingouin as pg

import seaborn as sns
import matplotlib as plt
%matplotlib inline

/Users/varya_hai/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/varya_hai/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
sns.set_theme(
    font_scale = 2,                         # размер шрифта
    style      = 'whitegrid',               # стиль поля графика
    rc         = {'figure.figsize':(20,10)}  # размер фигуры
)

# Задание


Представьте, что вы работаете аналитиком в очень крупной компании по доставке пиццы над приложением для курьеров (да, обычно в таких компаниях есть приложение для курьеров и отдельно приложение для потребителей).

У вас есть несколько ресторанов в разных частях города и целый штат курьеров. Но есть одна проблема — к вечеру скорость доставки падает из-за того, что курьеры уходят домой после рабочего дня, а количество заказов лишь растет. Это приводит к тому, что в момент пересмены наша доставка очень сильно проседает в эффективности. 

Наши data scientist-ы придумали новый алгоритм, который позволяет курьерам запланировать свои последние заказы перед окончанием рабочего дня так, чтобы их маршрут доставки совпадал с маршрутом до дома. То есть, чтобы курьеры доставляли последние свои заказы за день как бы "по пути" домой. 

Вы вместе с командой решили раскатить A/B тест на две равные группы курьеров. Часть курьеров использует старый алгоритм без опции "по пути", другие видят в своем приложении эту опцию и могут ее выбрать. Ваша задача – проанализировать данные эксперимента и помочь бизнесу принять решение о раскатке новой фичи на всех курьеров.


In [3]:
exp_lesson = pd.read_csv('/Users/varya_hai/Karpov.courses/4 experiment_lesson_4.csv')

In [4]:
exp_lesson.describe()

,order_id,delivery_time
count,20196.000000,20196.000000
mean,11103.503565,42.054169
std,5835.424404,10.382196
min,1000.000000,10.000000
25%,6048.750000,35.000000
50%,11109.500000,42.000000
75%,16158.250000,49.000000
max,21207.000000,83.000000


In [5]:
exp_lesson.head()

,order_id,delivery_time,district,experiment_group
0,3159,44.0,Purlieus,control
1,10426,51.0,Purlieus,control
2,9155,46.0,Purlieus,control
3,10699,34.0,Purlieus,control
4,10061,43.0,Purlieus,control


In [6]:
exp_lesson.shape

(20196, 4)

изменилась ли скорость доставки у тестовой группы? уменьшилась ли она?

In [7]:
control = exp_lesson.query('experiment_group=="control"')[['experiment_group','delivery_time']]
test = exp_lesson.query('experiment_group=="test"')[['experiment_group','delivery_time']]

In [8]:
st.normaltest(control.delivery_time) # pvalue>0.5 - нормальное распределение

NormaltestResult(statistic=0.25349755437872584, pvalue=0.8809549607809654)

In [9]:
st.normaltest(test.delivery_time) # pvalue<0.5 - НЕнормальное распределение

NormaltestResult(statistic=7.024157092641915, pvalue=0.02983483685333173)

In [10]:
control.delivery_time.std()

9.990016960246372

In [11]:
test.delivery_time.std()

9.883308351129061

In [12]:
control.delivery_time.median() - test.delivery_time.median()

6.0

In [13]:
st.ttest_ind(control.delivery_time,test.delivery_time) # получается, тесты контроля различаются!

Ttest_indResult(statistic=43.03582663183714, pvalue=0.0)

In [14]:
st.mannwhitneyu(control.delivery_time,test.delivery_time) # проверяет, с какой вероятностью случайное значение из одной группы
                                                          # превышает случайно взятое значение из другой группы

                                                          # в нашем случае, получилось, что группы различаются

MannwhitneyuResult(statistic=67984508.0, pvalue=0.0)

In [15]:
(1 - test.delivery_time.mean()/control.delivery_time.mean()) * 100 

13.354653121660808

In [5]:
books = pd.read_csv('/Users/varya_hai/Downloads/books_prep.csv')
books.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre,User Rating (Round),Price (Above Average)
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction,5,No
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction,5,Yes
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction,5,Yes
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction,5,No
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction,5,No


In [10]:
print(round(st.levene(books[books["Price (Above Average)"]=="Yes"]['User Rating'],books[books["Price (Above Average)"]=="No"]['User Rating']).pvalue,2))

0.1


In [16]:
print(round(st.f_oneway(books[books["User Rating (Round)"]==5].Reviews,books[books["User Rating (Round)"]==4].Reviews,books[books["User Rating (Round)"]==3].Reviews).pvalue,2))

0.3


In [17]:
books['User Rating (Round)'].unique()

array([5, 4, 3])